In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2025-10-17 11:13 ajna         INFO     Configuração de log efetuada


2025-10-17 11:13:55,908 ajna         INFO     Configuração de log efetuada


In [3]:
%load_ext autoreload 
%autoreload 2
engine = create_engine(SQL_URI)

# Recuperar informações de REDEX e OPERADORES

In [4]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO
LISTA_REDEX = "('8931309', '8933204', '8931404')" # Redex da Localfrio/Movecta, Redex da Santos Brasil Clia e Redex da DPW/EMBRAPORT
FILTRO_LIKE_REDEX = "'89327%'"  # Abrange todos os outros Redex santistas

from datetime import datetime, timedelta

agora = datetime.now()
ha_X_dias = agora - timedelta(days=7)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_X_dias.strftime('%Y-%m-%d %H:%M:%S')

In [5]:
# SQL para entrada_operadores (alias e)
sql_entrada_operadores = f"""
SELECT 
    id, placa, placaSemirreboque, numeroConteiner, codigoRecinto, dataHoraOcorrencia, operacao, direcao, 
    cnpjTransportador, cpfMotorista, nomeMotorista, listaNfe, numeroConhecimento, vazioConteiner
FROM apirecintos_acessosveiculo
WHERE 
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'E'
    AND codigoRecinto IN {LISTA_OPERADORES}
    AND numeroConteiner !='' 
    AND vazioConteiner = 0 
"""

# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'E' é porque queremos a Entrada no Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)
# Este 'C' é porque é registro de acesso (entrada) e não agendamento
# Este '0' indica que o conteiner chegou estufado (cheio) ao Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)

In [6]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [7]:
# SQL para pesagem_redex - pesagem com recintos Redex (estufadores)
sql_pesagem_redex = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [8]:
# SQL com JOIN para filtrar saidas do REDEX seguidas de entrada no Terminal Exportador (Operador portuário)
sql_saida_redex = f"""
SELECT DISTINCT
    c.id,
    c.placa,
    c.placaSemirreboque,
    c.numeroConteiner,
    c.vazioConteiner,
    c.codigoRecinto,
    c.cpfMotorista,
    c.nomeMotorista,
    c.dataHoraOcorrencia,
    c.direcao
FROM apirecintos_acessosveiculo AS c
JOIN apirecintos_acessosveiculo AS e
    ON e.numeroConteiner = c.numeroConteiner 
    AND e.placaSemirreboque = c.placaSemirreboque
    AND e.dataHoraOcorrencia >= c.dataHoraOcorrencia
    AND e.direcao = 'E'
    AND e.operacao = 'C'
    AND (
        (e.codigoRecinto NOT IN {LISTA_REDEX} AND e.codigoRecinto NOT LIKE {FILTRO_LIKE_REDEX})
        OR e.codigoRecinto = '8931404'
    )
    AND e.codigoRecinto != c.codigoRecinto
WHERE 
    c.dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND c.operacao = 'C'
    AND c.direcao = 'S'
    AND (c.codigoRecinto IN {LISTA_REDEX} OR c.codigoRecinto LIKE {FILTRO_LIKE_REDEX})
    AND c.numeroConteiner != ''
"""
# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'S' é porque queremos a Saida do Redex estufador
# Este 'C' é porque é registro de acesso (saída) e não agendamento
# A tabela c. é a tabela de ACESSO DE SAÍDA do REDEX no API RECINTOS
# A tabela e. é a tabela de ACESSO DE ENTRADA no TERMINAL EXPORTADOR (Operador Portuário)
# A lógica é que o caminhão com conteiner SAI DO REDEX ANTES DE ENTRAR NO TERMINAL EXPORTADOR (por isso o maior igual)
# Atenção! A DPW/EMBRAPORT pode operar como REDEX e envia carga para a BTP como pode receber carga de um REDEX e operar como Terminal exportador
# Este código contempla estes dois casos da DPW/EMBRAPORT

In [9]:
# Execução das consultas com parâmetros
df_entrada_operadores = pd.read_sql(
    text(sql_entrada_operadores), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [10]:
df_entrada_operadores

,id,placa,placaSemirreboque,numeroConteiner,codigoRecinto,dataHoraOcorrencia,operacao,direcao,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento,vazioConteiner
0,13030706,DPC9B40,TKB8F51,GCNU1384238,8931318,2025-10-10 11:31:32,C,E,02077058000169,65925807472,LAURILIO BORGES DE LIMA,35251053009825000133550070006843801026316983,,0
1,13030708,HOQ6F24,DBP4301,TEMU1869381,8931318,2025-10-10 11:33:28,C,E,32256956000145,35737519899,THILYE CALDAS SANTANA DE SOUZA,,,0
2,13030712,DVS1I68,CUA5F95,GCNU1339202,8931318,2025-10-10 11:37:29,C,E,02077058000169,37415733886,JEFEERSON DE SOUZA FELIX RODRIGUES,35251053009825000133550070006843801026316983,,0
3,13030713,BTT8F49,GAA2B04,MSDU7866466,8931318,2025-10-10 11:38:55,C,E,01896217000194,25601886865,EVERALDO DE SOUZA SANTOS,,,0
4,13030718,CUA4J91,CUA5944,ACLU2792117,8931318,2025-10-10 11:45:53,C,E,02077058000169,27769375800,MARCIO JOSE CARDOSO DOS SANTOS,35251053009825000133550070006843801026316983,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22148,13166537,SUU2J78,CZC2947,MSKU5044985,8931404,2025-10-16 23:55:18,C,E,51485274000159,32655794842,GILVAN NASCIMENTO RIBEIRO,43251000253137000409550010006958941961258886,,0
22149,13166542,AAW2B18,TJV4D17,FCIU7345494,8931404,2025-10-16 23:55:24,C,E,43388833000184,37339188860,RONALDO LUIZ BARROS SOUSA,,,0
22150,13166544,LPA8318,BXS8A26,MRSU7477373,8931404,2025-10-16 23:55:51,C,E,49871560000100,37275802830,ISAAC ALVES DA SILVA,35251059179838000218550040000999221497073758,,0
22151,13166549,GWI8E56,DPC9C36,DRYU9083225,8931404,2025-10-16 23:57:41,C,E,05700665000121,06220616820,SERGIO FIRMINO DOS SANTOS,35250955249627000415550010000459501001245483,,0


In [11]:
df_saida_redex = pd.read_sql(
    text(sql_saida_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [12]:
df_saida_redex

,id,placa,placaSemirreboque,numeroConteiner,vazioConteiner,codigoRecinto,cpfMotorista,nomeMotorista,dataHoraOcorrencia,direcao
0,13029396,HOM6694,EFU5433,SEGU2600180,0,8931309,04489579810,JOSE ALMEIDA DE MENEZES,2025-10-10 14:03:16,S
1,13029365,NCP8J86,TLT3F49,ZIMU3084832,0,8931309,28938218805,RODRIGO SEABRA DA SILVA,2025-10-10 15:10:23,S
2,13029493,JBW3J31,NXW2780,MSDU4752557,0,8931309,13401504843,STANLEY SILVA DE OLIVEIRA,2025-10-10 18:27:12,S
3,13029539,DPB7G49,DBC0H63,MRKU6690972,0,8931309,30193136864,DENNIS FERNANDO PASCHOAL,2025-10-11 00:44:37,S
4,13029671,HOM6694,EFU5433,MRKU7593025,0,8931309,04489579810,JOSE ALMEIDA DE MENEZES,2025-10-11 05:49:19,S
...,...,...,...,...,...,...,...,...,...,...
6059,13151641,MFY1756,DAO3I46,HLBU1307085,0,8933204,36216699814,NATALIA CARDOSO DOS SANTOS,2025-10-15 22:02:03,S
6060,13179487,CNR1J79,BSF4F24,INKU6192113,0,8933204,35177073840,SIDNEY DO NASCIMENTO CHAVES,2025-10-16 20:49:23,S
6061,13179511,KXC4F37,TIS2F42,MRSU5301560,0,8933204,44316350846,SAMUEL SOUZA SANTANA,2025-10-16 20:56:31,S
6062,13179496,TBS8H21,ESU7J47,TCNU1826348,0,8933204,24648743881,MARCIO ROGERIO DI PARDO,2025-10-16 21:40:03,S


In [13]:
df_saidas_entradas = pd.merge(df_entrada_operadores, df_saida_redex, how='inner', on=['placaSemirreboque', 'numeroConteiner'] , suffixes=('_E_Operador', '_S_REDEX'))
# Este 'inner' faz com que a tabela final obtida saidas_entradas só tenham dados que tem em Entrada Operador e Saída Rede
# Dados que não tenham ligação correspondente (são nulos) NÃO aparecerão
# Se você quiser que os não correspondentes também apareçam troque INNER por LEFT

In [14]:
df_saidas_entradas['transitTime'] = (df_saidas_entradas['dataHoraOcorrencia_E_Operador'] - 
                                     df_saidas_entradas['dataHoraOcorrencia_S_REDEX']).dt.total_seconds() / 3600


In [15]:
# df_saidas_entradas

In [16]:
#!pip install tqdm
from tqdm.notebook import tqdm

def buscar_id_fs_files(collection, numero_conteiner, data_ocorrencia):
    filtro = {
        'metadata.numeroinformado': numero_conteiner,
        'metadata.dataescaneamento': {
            '$gte': data_ocorrencia - timedelta(hours=3),
            '$lte': data_ocorrencia + timedelta(hours=6)
        }
    }
    resultado = collection.find_one(filtro, {'_id': 1})
    if resultado:
        return resultado['_id']
    return None

# Iterar sobre as linhas do DataFrame e aplicar busca
def mapear_ids_mongo(collection, df):
    ids_fs_files = []
    cont_sucesso = 0
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Buscando ids no MongoDB"):
        numero = row['numeroConteiner']
        data = row['dataHoraOcorrencia_E_Operador']
        id_fs = buscar_id_fs_files(collection, numero, data)
        ids_fs_files.append(id_fs)
        cont_sucesso += 1 if id_fs else 0
    df['fs_files_id'] = ids_fs_files
    print(f'Registros: {len(df)}, imagens: {cont_sucesso}')

In [17]:
mapear_ids_mongo(db['fs.files'], df_saidas_entradas)

Buscando ids no MongoDB:   0%|          | 0/6014 [00:00<?, ?it/s]

Registros: 6014, imagens: 2752


In [18]:
# df_saidas_entradas.transitTime.describe()
df_saidas_entradas.rename(columns={'placa': 'placa', 'placaSemirreboque': 'placa do semireboque', 'numeroConteiner': 'contêiner', 'vazioConteiner' : 'conteinervazio',
                         'codigoRecinto_S_REDEX': 'Redex', 'dataHoraOcorrencia_S_REDEX': 'Saída REDEX','placaSemirreboque': 'placa do semireboque',
                         'codigoRecinto_E_Operador': 'Operador', 'dataHoraOcorrencia_E_Operador': 'Entrada Operador',
                         'transitTime': 'transitTime', 'fs_files_id': '_id'}, inplace=True)
df_saidas_entradas.drop(['operacao', 'direcao_E_Operador', 'listaNfe', 'numeroConhecimento', 'direcao_S_REDEX'], axis=1)
df_saidas_entradas

,id_E_Operador,placa_E_Operador,placa do semireboque,contêiner,Operador,Entrada Operador,operacao,direcao_E_Operador,cnpjTransportador,cpfMotorista_E_Operador,...,id_S_REDEX,placa_S_REDEX,vazioConteiner_S_REDEX,Redex,cpfMotorista_S_REDEX,nomeMotorista_S_REDEX,Saída REDEX,direcao_S_REDEX,transitTime,_id
0,13030767,BXF4E94,ECT3E83,SEGU2932158,8931318,2025-10-10 13:18:35,C,E,02077058000169,04105523805,...,13076698,BXF4E94,0,8932797,04105523805,ANTONIO JOSE DOS REIS NETO,2025-10-10 12:09:29,S,1.151667,68e9555fcca6eeddad80fd6e
1,13030772,DPC9932,DPC9D37,TRHU2772379,8931318,2025-10-10 13:24:54,C,E,02077058000169,91308658400,...,13076686,DPC9932,0,8932797,91308658400,FRANCISCO RICARDO DE SOUSA ANDRADE,2025-10-10 11:41:07,S,1.729722,68e9555fcca6eeddad80fd76
2,13030773,DPB9628,DPC9336,GCNU1359209,8931318,2025-10-10 13:28:16,C,E,02077058000169,19752170803,...,13076702,DPB9628,0,8932797,19752170803,MARCELO MORGADO,2025-10-10 12:15:51,S,1.206944,68e9555fcca6eeddad80fd6a
3,13030771,DPC9006,DVS1H43,TCLU3339660,8931318,2025-10-10 13:29:50,C,E,02077058000169,30792294874,...,13076685,DPC9006,0,8932797,30792294874,CLAYTON MACHADO DE CARVALHO,2025-10-10 11:36:27,S,1.889722,68e9555fcca6eeddad80fd72
4,13030776,DVS1D58,CUA5F93,GCNU1342309,8931318,2025-10-10 13:38:52,C,E,02077058000169,26982970850,...,13076704,DVS1D58,0,8932797,26982970850,FABIO SANTOS DE JESUS,2025-10-10 12:19:51,S,1.316944,68e95568cca6eeddad80ff34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6009,13166488,GOY7F34,CFI0J89,MRSU5674455,8931404,2025-10-16 23:33:56,C,E,43388833000184,00364485590,...,13172871,GOY7F34,0,8932759,00364485590,GIVALDO NASCIMENTO SANTOS,2025-10-16 22:38:20,S,0.926667,None
6010,13166516,DJC3764,CUF1997,TCLU5862053,8931404,2025-10-16 23:41:31,C,E,05700665000121,05631854813,...,13173733,DJC3764,0,8932762,05631854813,ZEVANIO DOS SANTOS,2025-10-16 22:26:02,S,1.258056,68f227acfbe8aee415926a88
6011,13166542,AAW2B18,TJV4D17,FCIU7345494,8931404,2025-10-16 23:55:24,C,E,43388833000184,37339188860,...,13172877,AAW2B18,0,8932759,37339188860,RONALDO LUIS BARROS DE SOUZA,2025-10-16 23:10:20,S,0.751111,68f227acfbe8aee415926a0d
6012,13166549,GWI8E56,DPC9C36,DRYU9083225,8931404,2025-10-16 23:57:41,C,E,05700665000121,06220616820,...,13173788,GWI8E56,0,8932762,06220616820,SERGIO FIRMINO DOS SANTOS,2025-10-16 23:08:53,S,0.813333,68f227abfbe8aee415926a03


In [19]:
from IPython.display import display, HTML

def get_outliers(df):
    mean = df['transitTime'].mean()
    std = df['transitTime'].std()
    outliers = df[(df['transitTime'] < mean - 1 * std) | (df['transitTime'] > mean + 1 * std)]
    return outliers

def generate_html(_id):
    url = f"https://ajna1.rfoc.srf/virasana/file?_id={_id}"
    img = f"https://ajna1.rfoc.srf/virasana/imagens_cmap?_id={_id}"
    return f'<a href="{url}"><img src="{img}" width="200"></a>'
    
outliers = get_outliers(df_saidas_entradas)[['placa do semireboque', 'contêiner', 'Redex', 'Saída REDEX',
                                  'Operador', 'Entrada Operador', 'transitTime', '_id']]

outliers['html'] = outliers['_id'].apply(generate_html)
display(HTML(outliers.to_html(escape=False)))

,placa do semireboque,contêiner,Redex,Saída REDEX,Operador,Entrada Operador,transitTime,_id,html
833,BHN9361,CMAU4393359,8931404,2025-10-11 01:04:31,8931356,2025-10-13 14:09:23,61.081111,None,
957,BXD9G36,CAAU5228760,8931404,2025-10-10 16:32:07,8931356,2025-10-13 18:23:00,73.848056,None,
971,DJG2H93,MSKU0878521,8931404,2025-10-10 20:03:21,8931356,2025-10-13 19:21:59,71.310556,None,
988,SWF3A00,CMAU6622578,8931404,2025-10-11 14:06:07,8931356,2025-10-13 20:22:39,54.275556,None,
1094,CDT7B25,MRKU6069203,8931404,2025-10-10 11:47:53,8931356,2025-10-14 04:34:23,88.775000,None,
1115,EVO4705,TRHU4829473,8932799,2025-10-11 09:21:54,8931356,2025-10-14 05:41:49,68.331944,None,
1149,ELO3G96,FFAU5381860,8931404,2025-10-10 16:45:06,8931356,2025-10-14 08:02:28,87.289444,None,
1201,JDD0H27,HASU4269843,8931404,2025-10-10 21:18:07,8931356,2025-10-14 10:31:28,85.222500,None,
1693,FEJ6F30,CMAU5709251,8931404,2025-10-13 05:45:09,8931356,2025-10-15 23:51:49,66.111111,None,
2061,BXE9H02,HAMU2948421,8931404,2025-10-15 19:49:00,8931356,2025-10-16 10:38:50,14.830556,None,


In [20]:
# Adicionar células clicáveis que levem ao Ajna
df_saidas_entradas['contêiner'] = df_saidas_entradas['contêiner'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/bhadrasana2/consulta_container?numerolote={x}", "{x}")'
)
df_saidas_entradas['_id'] = df_saidas_entradas['_id'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/virasana/file?_id={x}", "{x}")'
)

# df_saidas_entradas

In [21]:
df_saidas_entradas.to_excel('saidas_entradas.xlsx', index=False)

In [22]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placaSemirreboque', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 7) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placaSemirreboque', 'numeroConteiner'], right_on=['placaSemirreboque', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 7) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) 
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'placaSemirreboque', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

KeyError: "['codigoRecinto_pesagem', 'codigoRecinto_saida'] not in index"